<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/openai_mistral_test_disitll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flash-attn --no-build-isolation -q

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load base + adapter (uses FP16 for efficiency)
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    dtype=torch.float16,
    device_map="auto",
    attn_implementation="flash_attention_2"  # Optional: For speed
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(base_model, "frankmorales2020/mistral-7b-gpt-oss-20b-distilled")

# Test prompt (from your dataset)
prompt = "### Instruction:\nWrite a Python function to compute Fibonacci numbers. Your solution must be memory-efficient and have a linear time complexity."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Write a Python function to compute Fibonacci numbers. Your solution must be memory-efficient and have a linear time complexity.

### Input Format:
The first line contains the total number of test cases. The next line contains an integer, N, denoting the nth Fibonacci number.

### Output Format:
Print the nth Fibonacci number.

### Sample Input:
1
8

### Sample Output:
21

### Explanation:
Fibonacci series: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...

### Solution:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    a = 0
    b = 1
    for i in range(2, n + 1):
        c = a + b
        a = b
        b = c
    return b

if __name__ == '__main__':
    n = int(input())
    print(fib(n))
